# Mini proyecto kaggle

In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [75]:
tf.config.list_physical_devices('GPU')


[]

In [62]:
hist = []

In [24]:
train = pd.read_csv('./data/train.csv')
evaluetion = pd.read_csv('./data/test.csv')



In [85]:
train.head(1)

,ID,Y,V1,V2,V3,V4,V5,V6,V7,V8,...,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90
0,1,2013,44.81144,0.83826,0,7.91314,10.94148,-0.04547,-15.16332,-10.47324,...,-8.44873,-230.30484,-40.94698,48.20025,-0.28694,155.76251,-56.23579,13.62599,123.92018,10.02845


## 1. Explorar data

In [ ]:
train.info()

Se cuenta cuantas clases hay por columna

In [5]:
len(train['Y'].value_counts())

74

In [ ]:
for c in train.columns:
    print(f'{c} = {len(train[c].value_counts())}')

> V3 podrias ser eliminado ????????????

## 2. Implementar NN

### 2.1 Preparación 

In [32]:
data = train.drop(columns=['ID', 'V3'])

X = data.drop(columns=['Y']).values
y = data['Y'].values

# División de los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalización de las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# Escala datos de validavión
data_eval= evaluetion.drop(columns=['ID', 'V3'])
data_eval = scaler.transform(data_eval)

c:\Users\facio\anaconda3\envs\tfproject\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [82]:
data_eval= evaluetion.drop(columns=['ID', 'V3'])

### 2.2 Construcción de modelo

### Descripción


El modelo de red neuronal Keras para construir una arquitectura de red más amplia para la regresión. A continuación, se describe los componentes del modelo y cómo se integra con un proceso de validación cruzada para evaluar su rendimiento:

#### 1. Estructura del Modelo:
**Capas Densas:***
Primera capa: 128 neuronas, función de activación ReLU, entrada ajustada al número de características de X.
Segunda capa: 64 neuronas, función de activación ReLU.
Tercera capa: 32 neuronas, función de activación ReLU.
Capa de salida: 1 neurona, función de activación lineal para la regresión.

**Regularización:**
BatchNormalization: Se aplica después de cada capa densa antes del dropout para normalizar las activaciones de la capa anterior.
Dropout: Se utiliza para reducir el sobreajuste al "desconectar" aleatoriamente un porcentaje de neuronas durante el entrenamiento. Los porcentajes son 0.2 después de la primera capa densa y 0.1 después de la segunda.

**Compilación del Modelo:**
Optimizador: Adam con una tasa de aprendizaje de 0.001.
Función de pérdida: Mean Squared Error (MSE), apropiada para problemas de regresión.
Métricas: Mean Absolute Error (MAE), proporciona una medida de la precisión del modelo en términos de error absoluto medio.


#### 2. Entrenamiento y Evaluación:
**Pipeline de Preprocesamiento:**
StandardScaler: Normaliza las características de entrada para tener media 0 y varianza 1, lo que a menudo mejora el rendimiento del entrenamiento en modelos de aprendizaje profundo.
KerasRegressor:
Envoltura que permite utilizar un modelo de Keras como un estimador en scikit-learn.
Configurado para entrenar durante 100 épocas con un tamaño de lote de 256.

**Validación Cruzada:**
KFold: Se usa para dividir los datos en 10 subconjuntos ("folds"), cada uno de los cuales se usa una vez como conjunto de prueba mientras que los otros 9 forman el conjunto de entrenamiento.

**Shuffle:** Los datos se mezclan antes de dividirlos para garantizar que los datos de prueba y entrenamiento sean aleatorios.
Random_state: Fijado para garantizar la reproducibilidad de la división de datos.

**Evaluación:**
Se utiliza cross_val_score para evaluar el pipeline en los diferentes folds con la métrica de neg_mean_squared_error. Esto proporciona una estimación robusta del error cuadrático medio del modelo en diferentes subconjuntos del dataset.

In [68]:
def wider_model():
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X.shape[1],)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.1),
        Dense(32, activation='relu'),
        BatchNormalization(),
        # Dropout(0.1),
        Dense(1, activation='linear') 
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    return model

In [78]:
from tensorflow.keras.callbacks import EarlyStopping

seed = 21
np.random.seed(seed)
estimators = []

estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=256, verbose=2)))
pipeline = Pipeline(estimators)


C:\Users\facio\AppData\Local\Temp\ipykernel_28544\1583466586.py:8: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=256, verbose=2)))


In [87]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold, scoring='neg_mean_squared_error')
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/100
274/274 - 2s - loss: 3998975.7500 - mae: 1999.7202 - 2s/epoch - 6ms/step
Epoch 2/100
274/274 - 1s - loss: 3946651.7500 - mae: 1986.5911 - 751ms/epoch - 3ms/step
Epoch 3/100
274/274 - 1s - loss: 3848265.7500 - mae: 1961.6606 - 733ms/epoch - 3ms/step
Epoch 4/100
274/274 - 1s - loss: 3709944.0000 - mae: 1926.0664 - 745ms/epoch - 3ms/step
Epoch 5/100
274/274 - 1s - loss: 3538522.2500 - mae: 1881.0220 - 733ms/epoch - 3ms/step
Epoch 6/100
274/274 - 1s - loss: 3340572.7500 - mae: 1827.6285 - 849ms/epoch - 3ms/step
Epoch 7/100
274/274 - 1s - loss: 3122222.0000 - mae: 1766.8633 - 782ms/epoch - 3ms/step
Epoch 8/100
274/274 - 1s - loss: 2889089.7500 - mae: 1699.5903 - 735ms/epoch - 3ms/step
Epoch 9/100
274/274 - 1s - loss: 2646308.2500 - mae: 1626.5800 - 733ms/epoch - 3ms/step
Epoch 10/100
274/274 - 1s - loss: 2398554.0000 - mae: 1548.5309 - 768ms/epoch - 3ms/step
Epoch 11/100
274/274 - 1s - loss: 2150063.2500 - mae: 1466.0841 - 948ms/epoch - 3ms/step
Epoch 12/100
274/274 - 1s - loss:

In [88]:
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: -73.14 (2.55) MSE


In [89]:
pipeline.fit(X,y)

Epoch 1/100
304/304 - 2s - loss: 3996832.7500 - mae: 1999.1849 - 2s/epoch - 6ms/step
Epoch 2/100
304/304 - 1s - loss: 3933000.0000 - mae: 1983.1482 - 874ms/epoch - 3ms/step
Epoch 3/100
304/304 - 1s - loss: 3813663.2500 - mae: 1952.8115 - 883ms/epoch - 3ms/step
Epoch 4/100
304/304 - 1s - loss: 3647470.7500 - mae: 1909.7643 - 986ms/epoch - 3ms/step
Epoch 5/100
304/304 - 1s - loss: 3443668.0000 - mae: 1855.6128 - 911ms/epoch - 3ms/step
Epoch 6/100
304/304 - 1s - loss: 3210978.0000 - mae: 1791.7914 - 848ms/epoch - 3ms/step
Epoch 7/100
304/304 - 1s - loss: 2957391.5000 - mae: 1719.5447 - 850ms/epoch - 3ms/step
Epoch 8/100
304/304 - 1s - loss: 2690152.2500 - mae: 1639.9706 - 850ms/epoch - 3ms/step
Epoch 9/100
304/304 - 1s - loss: 2415782.0000 - mae: 1554.0439 - 885ms/epoch - 3ms/step
Epoch 10/100
304/304 - 1s - loss: 2140147.5000 - mae: 1462.6500 - 848ms/epoch - 3ms/step
Epoch 11/100
304/304 - 1s - loss: 1868508.8750 - mae: 1366.6162 - 886ms/epoch - 3ms/step
Epoch 12/100
304/304 - 1s - loss:

Pipeline(steps=[('standardize', StandardScaler()),
                ('mlp',
                 <keras.wrappers.scikit_learn.KerasRegressor object at 0x0000014BAA1FF370>)])

In [ ]:
# Entrenamiento del modelo
history = model.fit(X_train, y_train, validation_split=0.3, epochs=150, batch_size=256, verbose=2, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)])

In [49]:
# Evaluación del modelo en el conjunto de prueba
test_loss, test_mae = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

487/487 [==============================] - 1s 1ms/step - loss: 1223.3323 - mae: 26.9070
Test Loss: 1223.332275390625, Test MAE: 26.90700340270996


# Predicción

In [90]:
def save_data(model, name):
    y_predict = model.predict(data_eval)

    df = evaluetion[['ID']]
    df['Y'] = y_predict
    df.to_csv(f'predict/{name}.csv', index=False)

save_data(pipeline, 'crossvalidation2')

c:\Users\facio\anaconda3\envs\tfproject\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


73/73 - 0s - 184ms/epoch - 3ms/step


C:\Users\facio\AppData\Local\Temp\ipykernel_28544\1292055614.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Y'] = y_predict


In [56]:
import pickle

def save_model(model, name):
    with open(f'./models/{name}.pkl', 'wb') as archivo_salida:
        pickle.dump(model, archivo_salida)

save_model(model, 'dropout-sample')

INFO:tensorflow:Assets written to: ram://ce156ec7-9e37-46b8-bda6-703c9aaca2d2/assets


In [93]:
hist.append({
    'desc': 'normalizada con 2 podas aumentando porcentaje de poda',
    'mae':  5.834270477294922,
    'mae_kaggle': None # pendiente
})

pd.DataFrame(hist)

,n,desc,mae,mae_kaggle
0,5,normalizada con una poda,6.186801,9.16731
1,NaN,normalizada con 2 podas,5.971045,NaN
2,NaN,normalizada con 2 podas aumentando porcentaje ...,5.843672,NaN
3,NaN,normalizada con 2 podas aumentando porcentaje ...,5.834270,NaN
